# Part 1

In [ ]:
import itertools


def processLine(input: str):
    stripped = input.strip()
    data = stripped.split()
    wanted = data[0][1:-1]

    odds = dict()
    for i in range(len(wanted)):
        if wanted[i] == "#":
            odds[str(i)] = True
        else:
            odds[str(i)] = False

    joltage = data[-1]
    buttonsUgly = data[1:-1]
    buttons = [blah[1:-1].split(",") for blah in buttonsUgly]
    # assume that a solution will only ever press a button once
    for i in range(1, len(buttons)):
        combinations = itertools.combinations(buttons, i)
        for combo in combinations:
            # print(combo)
            buttonState = {str(i): False for i in range(len(wanted))}
            for button in combo:
                for light in button:
                    buttonState[light] = not buttonState[light]
                if odds == buttonState:
                    return len(combo)
    return 0


def process(name: str):
    count = 0
    with open(name, "r") as f:
        for line in f:
            count += processLine(line)

    print(count)

In [ ]:
process("small.txt")

7


In [ ]:
process("input.txt")

469


In [ ]:
# Part 2

In [ ]:
import itertools
import random


def process2Line(joltage: list[int], buttons: list[list[int]], count):
    # print(joltage, buttons)
    maxSwitch = max(joltage)

    chosenLight = joltage.index(maxSwitch)
    filteredButtons = [button for button in buttons if chosenLight in button]

    possibilities = itertools.combinations_with_replacement(filteredButtons, maxSwitch)

    for p in possibilities:
        counts = [0 for i in range(len(joltage))]
        for lights in p:
            for light in lights:
                counts[int(light)] += 1
        remaining = [joltage[i] - counts[i] for i in range(len(joltage))]
        buttonCounts = [0 for b in buttons]
        for b in p:
            buttonCounts[buttons.index(b)] += 1
        if all(r == 0 for r in remaining):
            print(f"pressed buttons: {buttonCounts}")
            return maxSwitch
        if any(r < 0 for r in remaining):
            # print(f'failed with {buttonCounts}')
            continue
        result = process2Line(remaining, buttons, maxSwitch)
        if result < 0:
            continue
        print(f"pressed {buttonCounts}")
        return count + result
    # print(joltage)
    return -1


def process2(name: str):
    count = 0
    with open(name, "r") as f:
        for line in f:
            print(".", end="")
            stripped = line.strip()
            data = stripped.split()
            joltage = [int(x) for x in data[-1][1:-1].split(",")]
            buttonsUgly = data[1:-1]
            buttonsStrings = [blah[1:-1].split(",") for blah in buttonsUgly]
            buttons = [[int(val) for val in b] for b in buttonsStrings]
            count += process2Line(joltage, buttons, 0)

    print(count)

In [ ]:
# process2('small.txt')
process2("input.txt")

.pressed buttons: [0, 0, 0, 19, 0, 0]
pressed [9, 0, 20, 0, 3, 20]
.

In [35]:
import itertools
import random
import numpy as np
from scipy.optimize import linprog


def process3Line(joltage: list[int], buttons: list[list[int]], count):
    # print(joltage, buttons)
    minSwitch = min([j for j in joltage if j != 0])
    zeroes = set([j for j in range(len(joltage)) if joltage[j] == 0])

    chosenLight = joltage.index(minSwitch)
    filteredButtons = [
        button
        for button in buttons
        if chosenLight in button and len(zeroes.intersection(set(button))) == 0
    ]

    possibilities = itertools.combinations_with_replacement(filteredButtons, minSwitch)

    for p in possibilities:
        counts = [0 for i in range(len(joltage))]
        for lights in p:
            for light in lights:
                counts[int(light)] += 1
        remaining = [joltage[i] - counts[i] for i in range(len(joltage))]
        buttonCounts = [0 for b in buttons]
        for b in p:
            buttonCounts[buttons.index(b)] += 1
        if all(r == 0 for r in remaining):
            print(f"pressed buttons: {buttonCounts}")
            return minSwitch

        result = process3Line(remaining, buttons, count + minSwitch)
        if result < 0:
            continue
        print(f"pressed {buttonCounts}")
        return result + count
    return -1


def process3(name: str):
    count = 0
    with open(name, "r") as f:
        for line in f:
            print(".", end="")
            stripped = line.strip()
            data = stripped.split()
            joltage = np.array([int(x) for x in data[-1][1:-1].split(",")])
            buttonsUgly = data[1:-1]
            buttonsStrings = [blah[1:-1].split(",") for blah in buttonsUgly]
            buttons = [[int(val) for val in b] for b in buttonsStrings]
            inputs = np.array(
                [
                    [1 if j in button else 0 for button in buttons]
                    for j in range(len(joltage))
                ]
            )
            c = [1 for i in range(len(buttons))]
            # print(c, inputs, joltage)
            res = linprog(
                c=c,
                A_eq=inputs,
                b_eq=joltage,
                method="highs",
                integrality=True,
            )
            print(res.fun)
            count += res.fun

    print(count)

In [36]:
process3("small.txt")

.10.0
.12.0
.11.0
33.0
